# Emerging Technologies – Problems 1–5

This notebook will contain solutions to Problems 1–5 for the Emerging Technologies assessment.

## Problem 1: Generating Random Boolean Functions

Given four Boolean inputs and one Boolean output, the task is to generate a function that is either constant (always the same output) or balanced (outputs True for exactly half of all possible inputs). The function must be randomly chosen from all such valid functions and returned as a callable.

**Background**

A Boolean function maps a set of Boolean inputs to a single Boolean output. A function is *constant* if it always returns the same value (always True or always False), regardless of input. A function is *balanced* if it returns True for exactly half of all possible input combinations and False for the other half. Distinguishing between constant and balanced functions is central to the Deutsch–Jozsa algorithm, as quantum algorithms can determine this global property with fewer queries than classical approaches.

**Design approach**

There are 2⁴ = 16 possible input combinations for four Boolean variables. To generate a constant function, we select a single output value (True or False) and return it for all inputs. For a balanced function, we randomly select exactly 8 of the 16 inputs to map to True, with the remainder mapping to False; this mapping can be stored as a set or lookup table. The function will be returned as a callable f(a, b, c, d) constructed from the chosen mapping.

In [ ]:
import random
import itertools

def random_constant_balanced():
    """
    Returns a randomly chosen Boolean function of four inputs that is guaranteed
    to be either constant or balanced (as required by Deutsch–Jozsa).
    """
    if random.choice([True, False]):
        # Constant function
        output_value = random.choice([False, True])
        def f(a, b, c, d):
            """Constant function: always returns {}.""".format(output_value)
            return bool(output_value)
        return f
    else:
        # Balanced function
        inputs = list(itertools.product([False, True], repeat=4))
        true_inputs = set(random.sample(inputs, 8))
        def f(a, b, c, d):
            """Balanced function: returns True for exactly 8 of 16 inputs."""
            a, b, c, d = bool(a), bool(b), bool(c), bool(d)
            return (a, b, c, d) in true_inputs
        return f

In [ ]:
# Sanity check (optional)
f = random_constant_balanced()
inputs = list(itertools.product([False, True], repeat=4))
true_count = sum(f(*inp) for inp in inputs)
print(f"True outputs: {true_count}")
assert true_count in {0, 8, 16}